In [2]:
import math
import queue
import numpy as np
from time import time

In [14]:
class puzzle:
    space=0
    def __init__(self,state,goal_state,path_cost):
        self.state=state
        self.goal_state=goal_state
        self.path_cost=path_cost
        
        self.action=None
        self.left_child=None
        self.right_child=None
        self.up_child=None
        self.down_child=None
        self.parent=None
        
        self.striker=0
        self.col=int(math.sqrt(len(state)))
        puzzle.space+=1
        self.net_cost=(self.path_cost+self.heuristic_cost())
        
        
        
    def Move_left(self,index):
        if (index % self.col)> 0:
            new_state=self.state.copy()
            new_state[index-1],new_state[index]=new_state[index],new_state[index-1]
            
            node=puzzle(new_state,self.goal_state,self.path_cost+1)
            self.left_child=node
            node.action='LEFT'
            node.parent=self
        
        
    def Move_right(self,index):
        if (index % self.col)< (self.col-1):
            new_state=self.state.copy()
            new_state[index+1],new_state[index]=new_state[index],new_state[index+1]
            
            node=puzzle(new_state,self.goal_state,self.path_cost+1)
            self.right_child=node
            node.action='RIGHT'
            node.parent=self

            
    def Move_up(self,index):
        if(index-self.col >= 0):
            new_state=self.state.copy()
            new_state[index-
                      self.col],new_state[index]=new_state[index],new_state[index-self.col]
            
            node=puzzle(new_state,self.goal_state,self.path_cost+1)
            self.up_child=node
            node.action='UP'
            node.parent=self
            
            
    def Move_down(self,index):
        if(index+self.col < len(self.state)):
            new_state=self.state.copy()
            new_state[index+self.col],new_state[index]=new_state[index],new_state[index+self.col]
            
            node=puzzle(new_state,self.goal_state,self.path_cost+1)
            self.down_child=node
            node.action='DOWN'
            node.parent=self
                    
        
    def Isgoal(self):
        status=False
        if self.state == self.goal_state:
            status=True
        return status
    
    
    def heuristic_cost(self):
        estimated_cost=0
        for num in range(1,len(self.state)):
            distance=abs(self.goal_state.index(num)-self.state.index(num))
            estimated_cost=estimated_cost+int(distance/3)+int(distance%3)
        return estimated_cost
        
    
    def Expand_state(self):
        for i in range(len(self.state)):
            if self.state[i]==0:
                self.striker=i
                
        self.Move_right(self.striker)
        self.Move_left(self.striker)
        self.Move_up(self.striker)
        self.Move_down(self.striker)
      
    
    def find_path(self,en):
        solution=[]
        steps=[]
        solution.append(self.state)
        temp=self
        while temp.parent != None:
            steps.append(temp.action)
            temp=temp.parent
            solution.append(temp.state)
            
        solution=solution[:-1]
        solution.reverse()
        steps.reverse()
        return solution,steps,puzzle.space,en    

In [15]:
def arrange_list(list1,list2):
    y=list(np.argsort(np.array(list2)))
    l1=[]
    for i in y:
        l1.append(list1[i])
    list2.sort()

    return l1,list2

def show_puzzle(path):
    for i in range(len(path)):
        col=int(math.sqrt(len(path[i])))
        c=0
        for j in range(col):
            for k in range(col):
                print(path[i][c],end=" ")
                c+=1
            print(end='\n')
        print(end='\n')


def A_star(initial_state,final_state):
    start_node = puzzle(initial_state,final_state,0)
    open_list=[]
    open_list_cost=[]
    if start_node.Isgoal():
        return start_node.find_path()
    open_list.append(start_node)
    open_list_cost.append(start_node.net_cost)
    close_list=[]
    en=0
    
    while (len(open_list)):
        open_list,open_list_cost = arrange_list(open_list,open_list_cost)
        node=open_list[0]
        cost=open_list_cost[0]
        
        open_list.pop(0)
        open_list_cost.pop(0)
        
        close_list.append(node.state)
        node.Expand_state()
        en+=1
        children=[]
        
        if node.left_child:
            children.append(node.left_child)
        if node.right_child:
            children.append(node.right_child)
        if node.up_child:
            children.append(node.up_child)
        if node.down_child:
            children.append(node.down_child)
            
        for c in children:
            if c.state not in close_list:
                if c.Isgoal():
                    return c.find_path(en)
                open_list.append(c)
                open_list_cost.append(c.net_cost)

                
def IDA_star(initial_state,final_state):
    start_node = puzzle(initial_state,final_state,0)
    open_list=[]
    open_list_cost=[]
    if start_node.Isgoal():
        return start_node.find_path()
    open_list.append(start_node)
    open_list_cost.append(start_node.net_cost)
    close_list=[]
    en=0
    global count
    count=0
    
    while (len(open_list)):
        open_list,open_list_cost = arrange_list(open_list,open_list_cost)
        node=open_list[0]
        cost=open_list_cost[0]
        
        open_list.pop(0)
        open_list_cost.pop(0)
        
        close_list.append(node.state)
        node.Expand_state()
        en+=1
        children=[]
        temp=[]
        
        if node.left_child:
            children.append(node.left_child)
        if node.right_child:
            children.append(node.right_child)
        if node.up_child:
            children.append(node.up_child)
        if node.down_child:
            children.append(node.down_child)
            
        for c in children:
            if c.net_cost>cost:
                temp.append(c.net_cost)
        
        if temp:
            threshold=min(temp)
            count+=len(temp)-1
        else:
            threshold=cost
            
        for c in children:
            if c.state not in close_list:
                if c.Isgoal():
                    return c.find_path(en)
                if c.net_cost<= threshold:
                    open_list.append(c)
                    open_list_cost.append(c.net_cost)

                

def BFS(initial_state,final_state):
    start_node = puzzle(initial_state,final_state,0)
    open_list=queue.Queue()
    if start_node.Isgoal():
        return start_node.find_path()
    open_list.put(start_node)
    close_list=[]
    en=0
    
    while not(open_list.empty()):
        node=open_list.get()
        close_list.append(node.state)
        node.Expand_state()
        en+=1
        children=[]
        
        if node.left_child:
            children.append(node.left_child)
        if node.right_child:
            children.append(node.right_child)
        if node.up_child:
            children.append(node.up_child)
        if node.down_child:
            children.append(node.down_child)
            
        for c in children:
            if c.state not in close_list:
                if c.Isgoal():
                    return c.find_path(en)
                open_list.put(c)
                
                
def DFS(initial_state,final_state):
    start_node = puzzle(initial_state,final_state,0)
    open_list=queue.LifoQueue() 
    if start_node.Isgoal():
        return start_node.find_path()
    open_list.put(start_node)
    close_list=[]
    en=0
    
    while not(open_list.empty()):
        node=open_list.get()
        close_list.append(node.state)
        node.Expand_state()
        en+=1
        children=[]
        
        if node.left_child:
            children.append(node.left_child)
        if node.right_child:
            children.append(node.right_child)
        if node.up_child:
            children.append(node.up_child)
        if node.down_child:
            children.append(node.down_child)
            
        for c in children:
            if c.state not in close_list:
                if c.Isgoal():
                    return c.find_path(en)
                open_list.put(c)

In [ ]:
# goal_state=[1, 2, 3,0]

# state=[[2,0,1,3],[2, 3, 1,0]]

In [20]:
goal_state=[1,2,3,
            8,0,4,
            7,6,5]

state=[[2, 8, 1,
        0, 4, 3,
        7, 6, 5],
    
        [1, 3, 4,
         8, 6, 2,
         7, 0, 5]]

In [23]:
count=0
for i in range(2):
    print('initial state')

    c=0
    count=0
    col=int(math.sqrt(len(state[i])))
    for j in range(col):
        for k in range(col):
            print(state[i][c],end=" ")
            c+=1
        print(end='\n')

    #algo_call
    puzzle.space=0
    t0=time()
    solution1,steps1,space1,EN1=A_star(state[i],goal_state)
    t1=time()-t0
    
    puzzle.space=0
    t0=time()
    solution2,steps2,space2,EN2=IDA_star(state[i],goal_state)
    t2=time()-t0
    space2-=count

    puzzle.space=0
    t0=time()
    solution3,steps3,space3,EN3=BFS(state[i],goal_state)
    t3=time()-t0

    puzzle.space=0
    t0=time()
    solution4,steps4,space4,EN4=DFS(state[i],goal_state)
    t4=time()-t0


    print('A*')
    print(f'time: {t1}')
    print(f'Used space: {space1}')
    print(f'Expanded node : {EN1}')
    print(f'Steps: {len(steps1)}')
    print(f'Steps: {steps1}\n')

    print('IDA*')
    print(f'time: {t2}')
    print(f'Used space: {space2}')
    print(f'Expanded node : {EN2}')
    print(f'Steps: {len(steps2)}')
    print(f'Steps: {steps2}\n')

    
    print('BFS')
    print(f'time: {t3}')
    print(f'Used space: {space3}')
    print(f'Expanded node : {EN3}')
    print(f'Steps: {len(steps3)}')
    print(f'Steps: {steps3}\n')

    print('DFS')
    print(f'time: {t4}')
    print(f'Used space: {space4}')
    print(f'Expanded node : {EN4}')
    print(f'Steps: {len(steps4)}')
    print(f'Steps: {steps4}\n')

initial state
2 8 1 
0 4 3 
7 6 5 
A*
time: 0.003993511199951172
Used space: 42
Expanded node : 15
Steps: 9
Steps: ['UP', 'RIGHT', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'DOWN']

IDA*
time: 0.0
Used space: 30
Expanded node : 15
Steps: 9
Steps: ['UP', 'RIGHT', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'DOWN']

BFS
time: 0.020006179809570312
Used space: 666
Expanded node : 244
Steps: 9
Steps: ['UP', 'RIGHT', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'DOWN']

DFS
time: 0.6774663925170898
Used space: 9868
Expanded node : 3514
Steps: 3443
Steps: ['DOWN', 'RIGHT', 'UP', 'UP', 'RIGHT', 'DOWN', 'DOWN', 'LEFT', 'UP', 'UP', 'RIGHT', 'DOWN', 'DOWN', 'LEFT', 'UP', 'UP', 'RIGHT', 'DOWN', 'DOWN', 'LEFT', 'UP', 'UP', 'RIGHT', 'DOWN', 'DOWN', 'LEFT', 'UP', 'UP', 'RIGHT', 'DOWN', 'LEFT', 'DOWN', 'RIGHT', 'UP', 'UP', 'LEFT', 'DOWN', 'DOWN', 'RIGHT', 'UP', 'UP', 'LEFT', 'DOWN', 'DOWN', 'RIGHT', 'UP', 'UP', 'LEFT', 'DOWN', 'DOWN', 'RIGHT', 'UP', 'UP', 'LEFT', 'DOWN', 'DOWN', 'RIGHT', 